## 1、计算论文文本内容重复率

In [ ]:
import os
import json
import pandas as pd
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# 使用改进的Jaccard相似度计算文本相似度
def calculate_jaccard_similarity(text1, text2):
    words1 = [word for word in jieba.cut(text1.replace(" ", "")) if word.strip()]
    words2 = [word for word in jieba.cut(text2.replace(" ", "")) if word.strip()]
    
    # 计算集合的交集和并集
    set1 = set(words1)
    set2 = set(words2)
    
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    return intersection / union if union != 0 else 0

# 使用改进的余弦相似度计算文本相似度
def calculate_cosine_similarity(text1, text2):
    corpus = [text1.replace(" ", ""), text2.replace(" ", "")]
    
    def jieba_tokenizer(text):
        return jieba.lcut(text.replace(" ", ""))
    
    # TF-IDF向量化文本
    vectorizer = TfidfVectorizer(tokenizer=jieba_tokenizer)
    tfidf_matrix = vectorizer.fit_transform(corpus)
    
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

# 计算字符级N-gram相似度
def calculate_ngram_similarity(text1, text2, n=2):
    # 构建文本语料库
    corpus = [text1.replace(" ", ""), text2.replace(" ", "")]
    
    def jieba_ngram_tokenizer(text):
        words = jieba.lcut(text.replace(" ", "")) 
        return [" ".join(words[i:i+n]) for i in range(len(words)-n+1)]
    
    # 词频统计向量化
    vectorizer = CountVectorizer(analyzer=jieba_ngram_tokenizer)
    count_matrix = vectorizer.fit_transform(corpus)
    
    return cosine_similarity(count_matrix[0:1], count_matrix[1:2])[0][0]

# 计算论文重复率的函数
def calculate_duplication_rate(paper_content, all_papers_content, paper_index):
    current_paper_text = "".join([item['text'] for item in paper_content if item['type'] == 'text'])
    
    max_similarity = 0
    total_similarity = 0
    num_comparisons = 0
    
    for i, other_paper in enumerate(all_papers_content):
        if i == paper_index:
            continue 
            
        other_paper_text = "".join([item['text'] for item in other_paper if item['type'] == 'text'])
        
        # 计算三种相似度指标
        jaccard_sim = calculate_jaccard_similarity(current_paper_text, other_paper_text)
        cosine_sim = calculate_cosine_similarity(current_paper_text, other_paper_text)
        ngram_sim = calculate_ngram_similarity(current_paper_text, other_paper_text, n=2)
        
        # N-gram权重更高
        similarity = (0.2 * jaccard_sim + 0.3 * cosine_sim + 0.5 * ngram_sim)
        
        max_similarity = max(max_similarity, similarity)
        total_similarity += similarity
        num_comparisons += 1
    
    # 计算平均相似度
    avg_similarity = total_similarity / num_comparisons if num_comparisons else 0
    
    # 综合最大值和平均值计算最终重复率
    duplication_rate = (0.7 * max_similarity + 0.3 * avg_similarity) * 100
    
    return duplication_rate

def main():
    os.makedirs(os.path.dirname('./data/result3.xlsx'), exist_ok=True)
    
    print("正在读取附件1的参赛队号与加密号映射...")
    attachment1 = pd.read_excel('./data/附件1.xlsx')
    team_id_mapping = dict(zip(attachment1['加密号'], attachment1['参赛队号']))
    print(f"成功加载 {len(team_id_mapping)} 个队伍信息")
    
    data_dir = "./data/transform/"

    paper_files = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith("_content_list.json"):
                paper_files.append(os.path.join(root, file))
    
    print(f"找到 {len(paper_files)} 个论文文件")
    
    all_papers_content = []
    paper_names = []
    team_ids = []
    
    # 加载所有论文内容
    for i, paper_file in enumerate(paper_files):  
        print(f"加载第 {i+1}/{len(paper_files)} 篇论文: {paper_file}")
        
        # 提取加密号
        file_name = os.path.basename(paper_file)
        encryption_code = file_name.split('_')[0]
        
        # 获取对应的参赛队号
        team_id = team_id_mapping.get(encryption_code, '未知')
        team_ids.append(team_id)
        
        with open(paper_file, 'r', encoding='utf-8') as f:
            try:
                paper_content = json.load(f)
                all_papers_content.append(paper_content)
                paper_names.append(os.path.relpath(paper_file, data_dir))
                print(f"成功加载论文，参赛队号: {team_id}")
            except json.JSONDecodeError as e:
                print(f"解析JSON文件失败: {paper_file}")
                print(f"错误信息: {str(e)}")
    
    # 计算重复率
    duplication_rates = []
    for i, paper_content in enumerate(all_papers_content):
        print(f"正在计算第 {i+1}/{len(all_papers_content)} 篇论文的重复率，参赛队号: {team_ids[i]}")
        duplication_rate = calculate_duplication_rate(paper_content, all_papers_content, i)
        duplication_rates.append({
            '参赛队号': team_ids[i],
            '文本重复率': duplication_rate
        })
        print(f"参赛队号 {team_ids[i]} 的重复率为: {duplication_rate:.2f}%")  
    df_duplication = pd.DataFrame(duplication_rates)
    try:
        with pd.ExcelWriter('./data/result3.xlsx', engine='openpyxl') as writer:
            df_duplication.to_excel(writer, sheet_name='文本重复率', index=False)  
        print("结果已成功写入！")
    except Exception as e:
        print(f"写入Excel文件时出错: {str(e)}")

if __name__ == "__main__":
    main()

## 2、查找雷同图片

In [4]:
import os
import json
import pandas as pd
from PIL import Image
import imagehash
import random
from tqdm import tqdm

# 计算单个图片的哈希值
def compute_image_hash(image_path, hash_size=16):
    try:
        image = Image.open(image_path).convert('RGB')
        
        phash = str(imagehash.phash(image, hash_size=hash_size))
        dhash = str(imagehash.dhash(image, hash_size=hash_size))
        avg_hash = str(imagehash.average_hash(image, hash_size=hash_size))
        
        # 组合哈希值
        combined_hash = phash + dhash + avg_hash
        
        return combined_hash
    except Exception as e:
        print(f"计算图片哈希值时出错 {image_path}: {e}")
        return "0" * (hash_size * hash_size // 4 * 3)

# 计算两个哈希值之间的相似度
def compute_hash_similarity(hash1, hash2):
    try:
        # 拆分组合哈希值
        hash_length = len(hash1) // 3
        phash1, dhash1, avghash1 = hash1[:hash_length], hash1[hash_length:2*hash_length], hash1[2*hash_length:]
        phash2, dhash2, avghash2 = hash2[:hash_length], hash2[hash_length:2*hash_length], hash2[2*hash_length:]
        
        # 分别计算三种哈希的汉明距离
        p_distance = bin(int(phash1, 16) ^ int(phash2, 16)).count('1')
        d_distance = bin(int(dhash1, 16) ^ int(dhash2, 16)).count('1')
        avg_distance = bin(int(avghash1, 16) ^ int(avghash2, 16)).count('1')
        
        # 取三种距离的加权平均
        max_distance = len(phash1) * 4  # 最大可能的汉明距离
        total_distance = 0.4 * p_distance + 0.4 * d_distance + 0.2 * avg_distance
        
        # 计算相似度（距离越小相似度越高）
        similarity = 1 - (total_distance / max_distance)
        return similarity
    except Exception as e:
        print(f"计算哈希相似度时出错: {e}")
        return 0

# 查找相似图片
def find_similar_images_sampled(all_papers_content, paper_names, team_ids, base_dir="./data/transform/", sample_size=2000):
    print("开始进行抽样式的相似图片查找...")
    
    image_info = []
    hash_size = 16  # 哈希尺寸设置为16x16
    
    print("开始收集图片信息并计算哈希值...")
    total_papers = len(all_papers_content)
    
    for paper_idx, paper_content in enumerate(tqdm(all_papers_content, desc="处理论文")):
        # 显示当前处理的论文
        paper_desc = f"处理第 {paper_idx+1}/{total_papers} 篇论文"
        
        for item_idx, item in enumerate(paper_content):
            if item['type'] == 'image':
                if os.path.isabs(item['img_path']):
                    img_path = item['img_path']
                else:
                    img_path = os.path.normpath(item['img_path'])
                if not os.path.exists(img_path):
                    print(f"\r{paper_desc} - 警告: 图片不存在: {img_path}")
                    continue
                
                # 计算图片的页内序号
                in_page_idx = sum(1 for x in paper_content[:item_idx] 
                               if x['type'] == 'image' and x['page_idx'] == item['page_idx'])
                # 计算图片哈希值
                image_hash = compute_image_hash(img_path, hash_size)
                
                # 保存完整的图片信息（包括哈希值）
                info = {
                    'paper_idx': paper_idx,
                    'team_id': team_ids[paper_idx],
                    'item_idx': item_idx,
                    'page_idx': item['page_idx'],
                    'img_path': img_path,
                    'in_page_idx': in_page_idx,
                    'hash': image_hash
                }
                image_info.append(info)
    
    total_images = len(image_info)
    print(f"共收集和处理了 {total_images} 张图片")
    
    if total_images == 0:
        print("没有找到图片，跳过比较")
        return []
    
    actual_sample_size = min(sample_size, total_images)
    print(f"将从 {total_images} 张图片中随机抽取 {actual_sample_size} 张进行相似度比较")
    
    # 随机抽样
    sampled_images = random.sample(image_info, actual_sample_size)
    
    similar_images = []
    similarity_threshold = 0.85  # 设置相似度阈值
    
    total_comparisons = (actual_sample_size * (actual_sample_size - 1)) // 2
    progress_bar = tqdm(total=total_comparisons, desc="比较图片相似度")
    
    for i in range(len(sampled_images)):
        img1 = sampled_images[i]
        paper_idx1 = img1['paper_idx']
        hash1 = img1['hash']
        
        for j in range(i+1, len(sampled_images)):
            img2 = sampled_images[j]
            paper_idx2 = img2['paper_idx']
        
            progress_bar.update(1)
            
            # 跳过同一篇论文的图片
            if paper_idx1 == paper_idx2:
                continue
            
            # 计算相似度
            hash2 = img2['hash']
            similarity = compute_hash_similarity(hash1, hash2)
            
            # 检查相似度是否超过阈值
            if similarity > similarity_threshold:
                print(f"\n发现相似图片!")
                print(f"  论文 {img1['team_id']} 第 {img1['page_idx']} 页, 序号 {img1['in_page_idx']}")
                print(f"  论文 {img2['team_id']} 第 {img2['page_idx']} 页, 序号 {img2['in_page_idx']}")
                print(f"  相似度: {similarity:.4f}")
                
                # 添加到结果列表
                similar_images.append({
                    '参赛队号': img1['team_id'],
                    '雷同图片所在页码': img1['page_idx'],
                    '雷同图片的页内序号': img1['in_page_idx']
                })
    progress_bar.close()
    return similar_images

# 对结果进行去重操作
def deduplicate_similar(df):
    if df.empty:
        return df
    deduplicated_df = df.drop_duplicates(
        subset=['参赛队号', '雷同图片所在页码', '雷同图片的页内序号'],
        keep='first'
    )
    return deduplicated_df

def main():
    os.makedirs(os.path.dirname('./data/result3.xlsx'), exist_ok=True)
    
    print("读取附件1的参赛队号和加密号映射...")
    attachment1 = pd.read_excel('./data/附件1.xlsx')
    team_id_mapping = dict(zip(attachment1['加密号'], attachment1['参赛队号']))
    print(f"成功加载 {len(team_id_mapping)} 个队伍信息")
    
    data_dir = "./data/transform/"

    paper_files = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith("_content_list.json"):
                paper_files.append(os.path.join(root, file))
    
    print(f"找到 {len(paper_files)} 个论文文件")
    
    # 加载所有论文的内容
    all_papers_content = []
    paper_names = []
    team_ids = []  # 存储队伍ID
    
    for i, paper_file in enumerate(paper_files):  
        print(f"加载第 {i+1}/{len(paper_files)} 篇论文: {paper_file}")
        
        # 提取加密号
        file_name = os.path.basename(paper_file)
        encryption_code = file_name.split('_')[0]
        
        # 找到对应的队伍ID
        team_id = team_id_mapping.get(encryption_code, '未知')
        team_ids.append(team_id)
        
        with open(paper_file, 'r', encoding='utf-8') as f:
            try:
                paper_content = json.load(f)
                all_papers_content.append(paper_content)
                paper_names.append(os.path.relpath(paper_file, data_dir))
                print(f"成功加载论文，参赛队号: {team_id}")
            except json.JSONDecodeError as e:
                print(f"解析JSON文件失败: {paper_file}")
    
    # 抽样查找相似图片
    similar_images = find_similar_images_sampled(all_papers_content, paper_names, team_ids, data_dir, sample_size=2000)
    
    df_images = pd.DataFrame(similar_images)
    df_images = deduplicate_similar(df_images)
    try:
        with pd.ExcelWriter('./data/result3.xlsx', mode='a', engine='openpyxl') as writer:
            print("写入雷同图片工作表...")
            if not df_images.empty:
                df_images.to_excel(writer, sheet_name='雷同图片', index=False)
            else:
                empty_images = pd.DataFrame(columns=['参赛队号', '雷同图片所在页码', '雷同图片的页内序号'])
                empty_images.to_excel(writer, sheet_name='雷同图片', index=False)
        
        print("结果已成功写入!")
    except Exception as e:
        print(f"写入Excel文件时出错: {str(e)}")

if __name__ == "__main__":
    main()

读取附件1的参赛队号和加密号映射...
成功加载 151 个队伍信息
找到 149 个论文文件
加载第 1/149 篇论文: ./data/transform/B1000\md\B1000_content_list.json
成功加载论文，参赛队号: 202190000314
加载第 2/149 篇论文: ./data/transform/B1135\md\B1135_content_list.json
成功加载论文，参赛队号: 202190000255
加载第 3/149 篇论文: ./data/transform/B1206\md\B1206_content_list.json
成功加载论文，参赛队号: 202190000562
加载第 4/149 篇论文: ./data/transform/B1242\md\B1242_content_list.json
成功加载论文，参赛队号: 202190000316
加载第 5/149 篇论文: ./data/transform/B1302\md\B1302_content_list.json
成功加载论文，参赛队号: 202190000503
加载第 6/149 篇论文: ./data/transform/B1357\md\B1357_content_list.json
成功加载论文，参赛队号: 202190000554
加载第 7/149 篇论文: ./data/transform/B1415\md\B1415_content_list.json
成功加载论文，参赛队号: 202190000501
加载第 8/149 篇论文: ./data/transform/B1472\md\B1472_content_list.json
成功加载论文，参赛队号: 202190000293
加载第 9/149 篇论文: ./data/transform/B1503\md\B1503_content_list.json
成功加载论文，参赛队号: 202190000555
加载第 10/149 篇论文: ./data/transform/B1585\md\B1585_content_list.json
成功加载论文，参赛队号: 202190000449
加载第 11/149 篇论文: ./data/transform/B1655\md

处理论文:  38%|███▊      | 57/149 [00:16<00:29,  3.07it/s]

计算图片哈希值时出错 .: [Errno 13] Permission denied: 'D:\\Code\\project\\taidibei_A'
计算图片哈希值时出错 .: [Errno 13] Permission denied: 'D:\\Code\\project\\taidibei_A'


处理论文:  42%|████▏     | 62/149 [00:17<00:28,  3.11it/s]

计算图片哈希值时出错 .: [Errno 13] Permission denied: 'D:\\Code\\project\\taidibei_A'


...


发现相似图片!
  论文 202190000403 第 4 页, 序号 0
  论文 202190000422 第 12 页, 序号 1
  相似度: 0.9820

发现相似图片!
  论文 202190000026 第 20 页, 序号 0
  论文 202190000576 第 20 页, 序号 0
  相似度: 0.9453

发现相似图片!
  论文 202190000503 第 9 页, 序号 2
  论文 202190000433 第 31 页, 序号 5
  相似度: 0.8875

发现相似图片!
  论文 202190000546 第 4 页, 序号 1
  论文 202190000555 第 9 页, 序号 0
  相似度: 0.9102

发现相似图片!
  论文 202190000258 第 11 页, 序号 0
  论文 202190000153 第 19 页, 序号 0
  相似度: 0.8961

发现相似图片!
  论文 202190000258 第 11 页, 序号 0
  论文 202190000471 第 30 页, 序号 0
  相似度: 0.8891

发现相似图片!
  论文 202190000258 第 11 页, 序号 0
  论文 202190000556 第 11 页, 序号 0
  相似度: 0.8750

发现相似图片!
  论文 202190000258 第 11 页, 序号 0
  论文 202190000293 第 21 页, 序号 0
  相似度: 0.9125

发现相似图片!
  论文 202190000079 第 16 页, 序号 2
  论文 202190000100 第 10 页, 序号 0
  相似度: 0.9305

发现相似图片!
  论文 202190000423 第 16 页, 序号 1
  论文 202190000300 第 19 页, 序号 0
  相似度: 1.0000

发现相似图片!
  论文 202190000423 第 16 页, 序号 1
  论文 202190000422 第 17 页, 序号 1
  相似度: 1.0000

发现相似图片!
  论文 202190000423 第 16 页, 序号 1
  论文 202190000414 第 16 页, 序号 

比较图片相似度: 100%|█████████▉| 1995781/1999000 [00:14<00:00, 150173.30it/s]


发现相似图片!
  论文 202190000153 第 19 页, 序号 0
  论文 202190000293 第 21 页, 序号 0
  相似度: 0.9227

发现相似图片!
  论文 202190000206 第 27 页, 序号 1
  论文 202190000213 第 7 页, 序号 1
  相似度: 0.9633

发现相似图片!
  论文 202190000206 第 27 页, 序号 1
  论文 202190000146 第 3 页, 序号 1
  相似度: 0.9180

发现相似图片!
  论文 202190000415 第 8 页, 序号 2
  论文 202190000568 第 3 页, 序号 0
  相似度: 0.8828

发现相似图片!
  论文 202190000415 第 8 页, 序号 2
  论文 202190000403 第 6 页, 序号 1
  相似度: 0.9320

发现相似图片!
  论文 202190000415 第 8 页, 序号 2
  论文 202190000080 第 26 页, 序号 0
  相似度: 0.8680

发现相似图片!
  论文 202190000315 第 5 页, 序号 1
  论文 202190000066 第 8 页, 序号 1
  相似度: 0.8844

发现相似图片!
  论文 202190000472 第 34 页, 序号 0
  论文 202190000432 第 13 页, 序号 5
  相似度: 0.8703

发现相似图片!
  论文 202190000537 第 7 页, 序号 0
  论文 202190000419 第 7 页, 序号 1
  相似度: 0.8930

发现相似图片!
  论文 202190000401 第 27 页, 序号 1
  论文 202190000471 第 30 页, 序号 0
  相似度: 0.8633

发现相似图片!
  论文 202190000401 第 27 页, 序号 1
  论文 202190000556 第 11 页, 序号 0
  相似度: 0.8680

发现相似图片!
  论文 202190000300 第 19 页, 序号 0
  论文 202190000422 第 17 页, 序号 1
  相似度

比较图片相似度: 100%|██████████| 1999000/1999000 [00:14<00:00, 142125.55it/s]



发现相似图片!
  论文 202190000414 第 16 页, 序号 2
  论文 202190000577 第 17 页, 序号 0
  相似度: 1.0000

发现相似图片!
  论文 202190000419 第 15 页, 序号 1
  论文 202190000537 第 15 页, 序号 1
  相似度: 1.0000
写入雷同图片工作表...
结果已成功写入!


## 3、查找雷同公式

In [5]:
import os
import json
import pandas as pd

# 查找相似公式
def find_similar_formulas(all_papers_content, paper_names, team_ids):
    print(f"开始查找相似公式...")
    
    # 存储所有相似公式
    all_similar_formulas = []
    
    # 首先收集所有公式信息
    formula_info = []
    for paper_idx, paper_content in enumerate(all_papers_content):
        print(f"处理第 {paper_idx+1}/{len(all_papers_content)} 篇论文的公式...")
        for item_idx, item in enumerate(paper_content):
            if item['type'] == 'equation' and 'text' in item:
                formula_info.append({
                    'paper_idx': paper_idx,
                    'item_idx': item_idx,
                    'page_idx': item['page_idx'],
                    'formula': item['text'],
                    # 页内序号仅统计同一页的equation项
                    'in_page_idx': sum(1 for x in paper_content[:item_idx] 
                                      if x['type'] == 'equation' 
                                      and x['page_idx'] == item['page_idx'])
                })
    
    print(f"共收集到 {len(formula_info)} 个公式，开始比较...")
    
    comparison_count = 0
    total_comparisons = len(formula_info) * (len(formula_info) - 1) // 2
    
    # 已处理的公式对，避免重复比较
    processed_pairs = set()
    
    for i, form1 in enumerate(formula_info):
        paper_idx1 = form1['paper_idx']
            
        for j in range(i+1, len(formula_info)): 
            form2 = formula_info[j]
            paper_idx2 = form2['paper_idx']
            
            # 跳过相同论文中的公式
            if paper_idx1 == paper_idx2:
                continue
            
            # 创建唯一标识符判断是否已比较过
            pair_id = tuple(sorted([paper_idx1, paper_idx2]))
            formula_pair = (form1['formula'].strip(), form2['formula'].strip())
            unique_id = (pair_id, formula_pair)
            
            if unique_id in processed_pairs:
                continue      
            processed_pairs.add(unique_id)
                
            comparison_count += 1
            if comparison_count % 1000 == 0:
                print(f"已完成 {comparison_count}/{total_comparisons} 次公式比较...")
                
            # 计算相似度（公式完全匹配）
            if form1['formula'].strip() == form2['formula'].strip() and len(form1['formula'].strip()) > 5:
                print(f"发现相似公式!")
                all_similar_formulas.append({
                    '参赛队号': team_ids[paper_idx1],
                    '雷同公式所在页码': form1['page_idx'],
                    '雷同公式的页内序号': form1['in_page_idx']
                })
                all_similar_formulas.append({
                    '参赛队号': team_ids[paper_idx2],
                    '雷同公式所在页码': form2['page_idx'],
                    '雷同公式的页内序号': form2['in_page_idx']
                })
    
    print(f"公式比较完成，共发现 {len(all_similar_formulas)//2} 对雷同公式（{len(all_similar_formulas)} 条记录）")
    return all_similar_formulas

# 对结果进行去重操作
def deduplicate_results(df):
    if df.empty:
        return df
    deduplicated_df = df.drop_duplicates(
        subset=['参赛队号', '雷同公式所在页码', '雷同公式的页内序号'],
        keep='first'
    )
    return deduplicated_df

def main():
    os.makedirs(os.path.dirname('./data/result3.xlsx'), exist_ok=True)

    # 读取附件1的参赛队号和加密号映射
    print("读取附件1的参赛队号和加密号映射...")
    attachment1 = pd.read_excel('./data/附件1.xlsx')
    team_id_mapping = dict(zip(attachment1['加密号'], attachment1['参赛队号']))
    print(f"成功加载 {len(team_id_mapping)} 个队伍信息")
    
    data_dir = "./data/transform/"
    paper_files = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith("_content_list.json"):
                paper_files.append(os.path.join(root, file))
    
    print(f"找到 {len(paper_files)} 个论文文件")
    
    # 加载所有论文内容
    all_papers_content = []
    paper_names = []
    team_ids = []  # 存储参赛队号
    
    for i, paper_file in enumerate(paper_files):  
        print(f"加载第 {i+1}/{len(paper_files)} 篇论文: {paper_file}")
        
        # 提取加密号
        file_name = os.path.basename(paper_file)
        encryption_code = file_name.split('_')[0]
        
        # 获取对应参赛队号
        team_id = team_id_mapping.get(encryption_code, '未知')
        team_ids.append(team_id)
        
        with open(paper_file, 'r', encoding='utf-8') as f:
            try:
                paper_content = json.load(f)
                all_papers_content.append(paper_content)
                paper_names.append(os.path.relpath(paper_file, data_dir))
                print(f"成功加载论文，参赛队号: {team_id}")
            except json.JSONDecodeError as e:
                print(f"解析JSON文件失败: {paper_file}")
                print(f"错误信息: {str(e)}")
    
    similar_formulas = find_similar_formulas(all_papers_content, paper_names, team_ids)

    similar_formulas = pd.DataFrame(similar_formulas)
    df_formulas = deduplicate_results(similar_formulas)  

    try:
        with pd.ExcelWriter('./data/result3.xlsx', mode='a', engine='openpyxl') as writer:
            if not df_formulas.empty:
                df_formulas.to_excel(writer, sheet_name='雷同公式', index=False)
            else: 
                empty_formulas = pd.DataFrame(columns=['参赛队号', '雷同公式所在页码', '雷同公式的页内序号'])
                empty_formulas.to_excel(writer, sheet_name='雷同公式', index=False)
        
        print("结果已成功写入！")
    except Exception as e:
        print(f"写入Excel文件时出错: {str(e)}")

if __name__ == "__main__":
    main()

读取附件1的参赛队号和加密号映射...
成功加载 151 个队伍信息
找到 149 个论文文件
加载第 1/149 篇论文: ./data/transform/B1000\md\B1000_content_list.json
成功加载论文，参赛队号: 202190000314
加载第 2/149 篇论文: ./data/transform/B1135\md\B1135_content_list.json
成功加载论文，参赛队号: 202190000255
加载第 3/149 篇论文: ./data/transform/B1206\md\B1206_content_list.json
成功加载论文，参赛队号: 202190000562
加载第 4/149 篇论文: ./data/transform/B1242\md\B1242_content_list.json
成功加载论文，参赛队号: 202190000316
加载第 5/149 篇论文: ./data/transform/B1302\md\B1302_content_list.json
成功加载论文，参赛队号: 202190000503
加载第 6/149 篇论文: ./data/transform/B1357\md\B1357_content_list.json
成功加载论文，参赛队号: 202190000554
加载第 7/149 篇论文: ./data/transform/B1415\md\B1415_content_list.json
成功加载论文，参赛队号: 202190000501
加载第 8/149 篇论文: ./data/transform/B1472\md\B1472_content_list.json
成功加载论文，参赛队号: 202190000293
加载第 9/149 篇论文: ./data/transform/B1503\md\B1503_content_list.json
成功加载论文，参赛队号: 202190000555
加载第 10/149 篇论文: ./data/transform/B1585\md\B1585_content_list.json
成功加载论文，参赛队号: 202190000449
加载第 11/149 篇论文: ./data/transform/B1655\md

## 4、计算最终论文的重复率

In [13]:
import os
import pandas as pd

def combine_and_calculate_final_rate(excel_path):
    try:
        duplication_sheet = pd.read_excel(excel_path, sheet_name='文本重复率')
        image_sheet = pd.read_excel(excel_path, sheet_name='雷同图片')
        formula_sheet = pd.read_excel(excel_path, sheet_name='雷同公式')

        image_counts = image_sheet.groupby('参赛队号').size().reset_index(name='雷同图片数量')
        formula_counts = formula_sheet.groupby('参赛队号').size().reset_index(name='雷同公式数量')

        # 合并与论文重复率
        combined_df = duplication_sheet.merge(image_counts, on='参赛队号', how='left').merge(formula_counts, on='参赛队号', how='left')
        combined_df['雷同图片数量'] = combined_df['雷同图片数量'].fillna(0).astype(int)
        combined_df['雷同公式数量'] = combined_df['雷同公式数量'].fillna(0).astype(int)

        combined_df['重复率'] = (
            0.8 * combined_df['文本重复率'] + 
            0.1 * combined_df['雷同图片数量'] + 
            0.1 * combined_df['雷同公式数量']
        )

        with pd.ExcelWriter(excel_path, mode='a', engine='openpyxl') as writer:
            # writer.book.remove(writer.book['文本重复率'])   # 删除原始文本重复率工作表
            combined_df[['参赛队号', '重复率']].to_excel(writer, sheet_name='重复率', index=False)
        print(f"错误：文件 '{excel_path}' 未找到")
    except Exception as e:
        print(f"处理文件时出错: {str(e)}")

def main():
    excel_path = './data/result3.xlsx'
    if not os.path.exists(excel_path):
        print(f"错误：文件 '{excel_path}' 不存在")
        return
    
    # 执行合并和计算
    combine_and_calculate_final_rate(excel_path)

if __name__ == "__main__":
    main()

错误：文件 './data/result3.xlsx' 未找到
